In [ ]:
from csv import DictReader

nodes = {}
edges = {}

def get_marseille_bureau (row):
    bureau = row['customs_office_source']
    if bureau == "Septeme":
        bureau = "Septemes"
    if bureau == "Marseille, Poids et Casse":
        bureau = "Marseille - Poids et Casse"
    if bureau == "Penne":
        bureau = "La Penne"
    if bureau == "":
        bureau = "Direction des Fermes de Marseille (pas de bureau spécifié)"
    bureau = "bureau des fermes " + bureau
    return bureau

def create_or_update_node (id, type, value):
    if id not in nodes:
        nodes[id] = {
            "id": id,
            "type": type,
            "weight": value
        }
    else:
        nodes[id]["weight"] += value

def create_or_update_edge (provenance, destination, value):
    footprint = "from " + provenance + " to " + destination
    if footprint not in edges:
        edges[footprint] = {
            "from": provenance,
            "to": destination,
            "weight": value
        }
    else:
        edges[footprint]["weight"] += value

with open('../../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        
        value = int(row['value'].split('.')[0] or 0)
        
        if \
                row['year'] == '1789' \
            and row['best_guess_region_prodxpart'] == '1' \
            :
            # Marseille is reporting
            if row['customs_region'] == 'Marseille':
                bureau = get_marseille_bureau(row)
                create_or_update_node(bureau, "Marseille", value)
                # Exports from Marseille
                if row['export_import'] == 'Exports':
                    if row['partner_grouping'] == 'France':
                        if row['partner_simplification'] == 'Marseille':
                            partner = "Direction des Fermes de Marseille (pas de bureau spécifié)"
                            create_or_update_node(partner, "Marseille", value)
                        else:
                            partner = "ports francs de France"
                            create_or_update_node(partner, "France hors Marseille", value)
                    else:
                        partner = "Monde"
                        create_or_update_node(partner, "Monde", value)
                    create_or_update_edge(bureau, partner, value)
                # Imports to Marseille
                elif row['export_import'] == 'Imports':
                    if row['partner_grouping'] == 'France':  
                        partner = "ports francs de France"
                        create_or_update_node(partner, "France hors Marseille", value)
                    else:
                        partner = "Monde"
                        create_or_update_node(partner, "Monde", value)
                    create_or_update_edge(partner, bureau, value)
            # Marseille is partner
            if row['partner_simplification'] == 'Marseille':
                # Marseille to Marseille
                if row['customs_region'] == 'Marseille':
                    reporter = get_marseille_bureau(row)
                    create_or_update_node(reporter, "Marseille", value)
                else:
                    reporter = "directions des Fermes hors Marseille"
                    create_or_update_node(reporter, "France hors Marseille", value)
                partner = "Direction des Fermes de Marseille (pas de bureau spécifié)"
                create_or_update_node(partner, "Marseille", value)
                # Exports to Marseille
                if row['export_import'] == 'Exports':
                    create_or_update_edge(reporter, partner, value)
                # Imports from Marseille
                elif row['export_import'] == 'Imports':
                    create_or_update_edge(partner, reporter, value)
import networkx as nx
from ipysigma import Sigma

graph = nx.DiGraph()

for id, props in nodes.items():
    graph.add_node(id)
    for key, val in props.items():
        graph.nodes[id][key] = val

for id, props in edges.items():
    provenance = props["from"]
    destination = props["to"]
    graph.add_edge(provenance, destination)
    for key, val in props.items():
        graph.edges[provenance, destination][key] = val
    graph.edges[provenance, destination]["label"] = str(int(props["weight"])) + " lt."
    
nx.write_gexf(
    graph, 
    "toflit18-bureaux-network.gexf",
    encoding='utf-8',
    prettyprint=True
)

In [ ]:
# essai mirror flows
from csv import DictReader

nodes = {}
edges = {}

def get_marseille_bureau (row):
    bureau = row['customs_office_source']
    if bureau == "Septeme":
        bureau = "Septemes"
    if bureau == "Marseille, Poids et Casse":
        bureau = "Marseille - Poids et Casse"
    if bureau == "Penne":
        bureau = "La Penne"
    if bureau == "":
        bureau = "Direction des Fermes de Marseille (pas de bureau spécifié)"
    bureau = "bureau des fermes " + bureau
    return bureau

def create_or_update_node (id, type, value):
    if id not in nodes:
        nodes[id] = {
            "id": id,
            "type": type,
            "weight": value
        }
    else:
        nodes[id]["weight"] += value

def create_or_update_edge (provenance, destination, flow_type, value):
    footprint = "from " + provenance + " to " + destination + " type " + flow_type
    if footprint not in edges:
        edges[footprint] = {
            "from": provenance,
            "to": destination,
            "flow_type": flow_type,
            "weight": value
        }
    else:
        edges[footprint]["weight"] += value

with open('../../data/toflit18_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        
        value = int(row['value'].split('.')[0] or 0)
        
        if \
                row['year'] == '1789' \
            and row['best_guess_region_prodxpart'] == '1' \
            :
            flow_type = row['export_import']
            # Marseille is reporting
            if row['customs_region'] == 'Marseille':
                bureau = get_marseille_bureau(row)
                create_or_update_node(bureau, "Marseille", value)
                # Exports from Marseille
                if flow_type == 'Exports':
                    if row['partner_grouping'] == 'France':
                        if row['partner_simplification'] == 'Marseille':
                            partner = "Direction des Fermes de Marseille (pas de bureau spécifié)"
                            create_or_update_node(partner, "Marseille", value)
                        else:
                            partner = "ports francs de France"
                            create_or_update_node(partner, "France hors Marseille", value)
                    else:
                        partner = "Monde"
                        create_or_update_node(partner, "Monde", value)
                    create_or_update_edge(bureau, partner, flow_type, value)
                # Imports to Marseille
                elif flow_type == 'Imports':
                    if row['partner_grouping'] == 'France':  
                        partner = "ports francs de France"
                        create_or_update_node(partner, "France hors Marseille", value)
                    else:
                        partner = "Monde"
                        create_or_update_node(partner, "Monde", value)
                    create_or_update_edge(partner, bureau, flow_type, value)
            # Marseille is partner
            if row['partner_simplification'] == 'Marseille':
                # Marseille to Marseille
                if row['customs_region'] == 'Marseille':
                    reporter = get_marseille_bureau(row)
                    create_or_update_node(reporter, "Marseille", value)
                else:
                    reporter = "directions des Fermes hors Marseille"
                    create_or_update_node(reporter, "France hors Marseille", value)
                partner = "Direction des Fermes de Marseille (pas de bureau spécifié)"
                create_or_update_node(partner, "Marseille", value)
                # Exports to Marseille
                if flow_type == 'Exports':
                    create_or_update_edge(reporter, partner, flow_type, value)
                # Imports from Marseille
                elif flow_type == 'Imports':
                    create_or_update_edge(partner, reporter, flow_type, value)

"""
for id1, edge1 in edges.items():
    for id2, edge2 in edges.items():
        if edge1["from"] == edge2["to"] or edge1["to"] == edge2["from"]:
            print("miroir", id1, id2)
"""
import networkx as nx
from ipysigma import Sigma

graph = nx.DiGraph()

for id, props in nodes.items():
    graph.add_node(id)
    for key, val in props.items():
        graph.nodes[id][key] = val

for id, props in edges.items():
    provenance = props["from"]
    destination = props["to"]
    graph.add_edge(provenance, destination)
    for key, val in props.items():
        graph.edges[provenance, destination][key] = val
    graph.edges[provenance, destination]["label"] = str(int(props["weight"])) + " lt."
    
nx.write_gexf(
    graph, 
    "toflit18-bureaux-network-with-mirrors.gexf",
    encoding='utf-8',
    prettyprint=True
)

In [ ]:
import networkx as nx
from ipysigma import Sigma

# Sigma(graph, node_color='type', background_color="black")